В данной работе необходимо предсказать общий обьем продаж для каждого товара и магазина в следующем месяце на основе данных временных рядов. Алгоритм для построения регрессионной модели является таким же, как и для первой лабораторной работы.

Для начала импортируем необходимые библиотеки

In [132]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)
import sys
import gc
import pickle
sys.version_info

In [133]:
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
cats = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')
train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test  = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')

In [134]:
items.head(10)

In [135]:
shops.head(10)

In [136]:
cats.head(10)

In [137]:
train.head(10)

In [138]:
test.head(10)

Рассмотрим данные, они включают следующие признаки: даты проданного товара, номер месяца, номер товаров и магазинов, номер представляющий пару магазин - товар, номер категории товара, количество проданных товаров, цена товара, наименование товара, наименование магазина, наименование категории товара. Так как пропущенных данных нету, рассмотрим их значения на графиках.

In [139]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=train.item_cnt_day)

In [140]:
plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price)

На графиках видны 2 сильно выделяющихся значения, для большего качества удалим их из общего количества

In [141]:
train = train[train.item_price<90000]
train = train[train.item_cnt_day<1001]

Также заменим значение признака цена товара, который меньше нуля на среднее значение цены

In [142]:
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

Исправим повторения в признаках названий магазинов заменив их номера

In [143]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

Для удобства данных приведем их строковые признаки к числовым. Заменим названия магазинов, на код города, в котором они располагаются, также разделим название категории товара на 2 категории и обе занесем в виде численных значений кодов соответствующих наименований категорий и подкатегорий товара.

In [144]:
#разделение проводим с помощью функции split
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
# для случая если признак состоит только из 1 серии(содержит в себе 1 часть изначально)
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)

In [145]:
shops.head(10)

In [146]:
cats.head(10)

In [147]:
test.describe(include="all")

С помощью функции intersection, найдем пересечение множеств (общие идентификаторы товара) для обучающей и тестовой выборок, при вычитании длинны тестовой из общего множества двух выборок найдем количество новых товаров тестовой выборки по сравнению с обучающей выборкой

In [148]:
len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))), len(list(set(test.item_id))), len(test)

Отсюда видно, что в наборе тестовых данных содержится 214200 значений Магазин-товар, а также 363 новых товара, значит для большинства элементов тестового набора данных целевое значение должно быть равно нулю. А обучающая выборка содержит пары, которые были проданы или возвращены. Из этого следует, что хорошим решением будет подсчитать ежемесячные продажи, а также добавить в них продажи для каждой пары в течение месяца, значения которых равны нулю. Тогда данные обучающей выборки будут аналогичны данным тестовой.

In [149]:
#добавляем в матрицу matrix признаки: номер магазина и товара по номерам месяцев
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
#преобразуем значения матрицы к значениям int    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)

создадим в обучающей выборке новый признак, как произведение количества проданных товаров и их цену, чтобы рассчитать значение выручки, затем выделим значения в диапазоне(0, 20). Таким образом мы сделаем обучающую целевую величину наиболее близкой к тестовому предсказанию.

In [150]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']

In [151]:
#вычисляем сумму количества проданных товаров в день(или находим количество проданных товаров в месяц) и присваиваем в данные matrix
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20)
                                .astype(np.float16))

Также добавим значения из тестовой выборки в матрицу

In [152]:
#приведем данные к типу int
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [153]:
#с помощью функции concat сцепляем матрицы по столбцам
matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True) # 34 месяца

In [154]:
#Добавляем признаки номера магазина, товара, номера категории товара с помощью функции merge, также приводим признаки к типу int
matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')
matrix['city_code'] = matrix['city_code'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)

In [155]:
#Вводим функцию, для нахождения количества проданных товаров за n месяцев
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [156]:
#добавим признаки количества товаров проданных за 1,2,3,6 и 12 месяцев
matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')

In [157]:
#Добавим признак среднего количества проданных товаров в месяц
group = matrix.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num'], how='left')
matrix['date_avg_item_cnt'] = matrix['date_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_avg_item_cnt')
matrix.drop(['date_avg_item_cnt'], axis=1, inplace=True)

In [158]:
#Добавим признак среднего количества проданных товаров за 1,2,3,6 и 12 месяцев
group = matrix.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_item_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_cnt'] = matrix['date_item_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_item_avg_item_cnt')
matrix.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)

In [159]:
#Добавим признак среднего количества проданных товаров для каждого магазина в n месяцев
group = matrix.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_shop_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_avg_item_cnt'] = matrix['date_shop_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1,2,3,6,12], 'date_shop_avg_item_cnt')
matrix.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)

In [160]:
#Добавим признак среднего количества проданных товаров для каждого номера категории за 1 месяц
group = matrix.groupby(['date_block_num', 'item_category_id']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_cat_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_category_id'], how='left')
matrix['date_cat_avg_item_cnt'] = matrix['date_cat_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_cat_avg_item_cnt')
matrix.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)

In [161]:
#Добавим признак среднего количества проданных товаров от номера категории для каждого магазина за 1 месяц
group = matrix.groupby(['date_block_num', 'shop_id', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = ['date_shop_type_avg_item_cnt']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'shop_id', 'type_code'], how='left')
matrix['date_shop_type_avg_item_cnt'] = matrix['date_shop_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_shop_type_avg_item_cnt')
matrix.drop(['date_shop_type_avg_item_cnt'], axis=1, inplace=True)

In [162]:
#Добавим признак среднего количества проданных товаров для каждого города за 1 месяц
group = matrix.groupby(['date_block_num', 'city_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_city_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'city_code'], how='left')
matrix['date_city_avg_item_cnt'] = matrix['date_city_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_city_avg_item_cnt')
matrix.drop(['date_city_avg_item_cnt'], axis=1, inplace=True)

In [163]:
#Добавим признак среднего количества проданных товаров каждого типа категории за 1 месяц
group = matrix.groupby(['date_block_num', 'type_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_type_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'type_code'], how='left')
matrix['date_type_avg_item_cnt'] = matrix['date_type_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_type_avg_item_cnt')
matrix.drop(['date_type_avg_item_cnt'], axis=1, inplace=True)

In [164]:
#Добавим признак среднего количества проданных товаров каждого подтипа категории за 1 месяц
group = matrix.groupby(['date_block_num', 'subtype_code']).agg({'item_cnt_month': ['mean']})
group.columns = [ 'date_subtype_avg_item_cnt' ]
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num', 'subtype_code'], how='left')
matrix['date_subtype_avg_item_cnt'] = matrix['date_subtype_avg_item_cnt'].astype(np.float16)
matrix = lag_feature(matrix, [1], 'date_subtype_avg_item_cnt')
matrix.drop(['date_subtype_avg_item_cnt'], axis=1, inplace=True)

Рассчитаем изменения цен за последние 6 месяцев

In [165]:
group = train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)

Добавим признак динамики выручки магазина в последнем месяце как разность между выручкой магазина и средней выручкой, деленную на среднюю выручку

In [166]:
group = train.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['shop_id'], how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1], 'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)

Добавим количество месяцев в признаки

In [167]:
matrix['month'] = matrix['date_block_num'] % 12

Добавим признак количества месяцев после старта продаж для пары магазин-товар и для номера товара отдельно

In [168]:
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')

Так как мы уже нашли отдельно все признаки с зависимостями количества месяцев меньше 12, можно удалить эти значения из признака номера месяца

In [169]:
matrix = matrix[matrix.date_block_num > 11]

Также заполним все пустые значения в признаках рассчитаных в определенное количество месяцев n

In [170]:
def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            if ('item_cnt' in col):
                df[col].fillna(0, inplace=True)         
    return df

matrix = fill_na(matrix)

Занесем все данные в отдельную таблицу pickle для удобства, удалим ненужные данные, файл test оставим для получения выходного файла отправки

In [171]:
matrix.to_pickle('data.pkl')
del matrix
del group
del items
del shops
del cats
del train

In [172]:
data = pd.read_pickle('data.pkl')

Занесем все признаки, которые будут рассматриваться в переменную data

In [173]:
data = data[[ 'date_block_num','shop_id','item_id','item_cnt_month','city_code','item_category_id','type_code','subtype_code','item_cnt_month_lag_1','item_cnt_month_lag_2',
    'item_cnt_month_lag_3','item_cnt_month_lag_6','item_cnt_month_lag_12','date_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_2',
    'date_item_avg_item_cnt_lag_3','date_item_avg_item_cnt_lag_6','date_item_avg_item_cnt_lag_12','date_shop_avg_item_cnt_lag_1','date_shop_avg_item_cnt_lag_2',
    'date_shop_avg_item_cnt_lag_3','date_shop_avg_item_cnt_lag_6','date_shop_avg_item_cnt_lag_12','date_cat_avg_item_cnt_lag_1','month','item_shop_first_sale',
    'item_first_sale','delta_price_lag']]

Для проверки тестовой выборки необходимо все 34 месяца, для самого набора данных проверок небходимо 33 месяца, для набора обучающих данных можно использовать от 0,3 величины данных, то есть 12-33 месяцев. Разбиваем данные на обучающий и тестовый набор, также удаляем целевой признак из X_train, X_valid и X_test. Обучающий набор будет использоваться для создания модели XGBoost, а тестовый набор будет использоваться, для того, чтобы сделать прогнозы, по которым мы можем оценить качество модели.

In [174]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
Y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
Y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

In [175]:
del data
gc.collect();

С помощью библиотеки xgboost построим модель линейной регрессии, которая использует метрику RMSE (квадратный корень из MSE метрики или среднеквадратичной ошибки), которая берет квадратный корень из усредненного отклонения от истинных значений в квадрате. Построим модель для обучающего и тестового набора и проведем мониторинг тестов эффективности для каждого набора.

In [176]:
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 10)

Делаем прогноз на основе модели для заданного диапазона из условия.

In [177]:
Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(X_test).clip(0, 20)

#Создаем таблицу для сохранения в виде выходного файла данных
submission = pd.DataFrame({
    "ID": test.index, 
    "item_cnt_month": Y_test
})

На основе модели построим зависимость важности признаков (каждый признак отображен с его весом в модели)

In [178]:
plot_features(model, (10,14))

Отсюда основными весовыми признаками являются: номер категории товара, название подтипа товара, месяц, номер товара, изменение цен за последние 6 месяцев, количество товаров проданных за 1 месяц и среднее количество проданных товаров за 1 месяц.

Сохраняем выходной файл прогноза в формате csv

In [179]:
submission.to_csv('xgb_submission.csv', index=False)
pickle.dump(Y_pred, open('xgb_train.pickle', 'wb'))
pickle.dump(Y_test, open('xgb_test.pickle', 'wb'))